First install dependencies:

pip install openmeteo-requests
pip install requests-cache retry-requests numpy pandas

In [1]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

In [2]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
responses = []
df = pd.DataFrame()

In [3]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"

params = {
	"latitude": 51.5085,
	"longitude": -0.1257,
	"start_date": "1980-01-01",
	"end_date": "2020-01-01",
	"daily": ["weather_code",
           "temperature_2m_mean",
           "temperature_2m_max",
           "temperature_2m_min",
           "precipitation_sum",
           "rain_sum",
           "snowfall_sum",
           "precipitation_hours",
           "wind_speed_10m_max",
           "wind_gusts_10m_max",
           "wind_direction_10m_dominant",
           "shortwave_radiation_sum"]
}

#coordinates = {'madrid':{'latitud':'40,4165', 'longitude':'-3,7026'},
#               'london':{'latitud':'51,5085', 'longitude':'-0,1257'},
#               'paris':{'latitud':'48,8534', 'longitude':'2,3488'},
#               'berlin':{'latitud':'52,5244', 'longitude':'13,4105'},
#               'brussels':{'latitud':'50,8505', 'longitude':'4,3488'},
#               'istanbul':{'latitud':'41,0138', 'longitude':'28,9497'}
#}

In [4]:
def get_weather_data():
    response = openmeteo.weather_api(url, params=params)
    return response

In [5]:
def get_weather_dataframe():
    daily_dataframe = pd.DataFrame()
    response = get_weather_data()
    daily = response[0].Daily()
    daily_weather_code = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()
    daily_temperature_2m_max = daily.Variables(2).ValuesAsNumpy()
    daily_temperature_2m_min = daily.Variables(3).ValuesAsNumpy()
    daily_precipitation_sum = daily.Variables(4).ValuesAsNumpy()
    daily_rain_sum = daily.Variables(5).ValuesAsNumpy()
    daily_snowfall_sum = daily.Variables(6).ValuesAsNumpy()
    daily_precipitation_hours = daily.Variables(7).ValuesAsNumpy()
    daily_wind_speed_10m_max = daily.Variables(8).ValuesAsNumpy()
    daily_wind_gusts_10m_max = daily.Variables(9).ValuesAsNumpy()
    daily_wind_direction_10m_dominant = daily.Variables(10).ValuesAsNumpy()
    daily_shortwave_radiation_sum = daily.Variables(11).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["weather_code"] = daily_weather_code
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
    daily_data["temperature_2m_max"] = daily_temperature_2m_max
    daily_data["temperature_2m_min"] = daily_temperature_2m_min
    daily_data["precipitation_sum"] = daily_precipitation_sum
    daily_data["rain_sum"] = daily_rain_sum
    daily_data["snowfall_sum"] = daily_snowfall_sum
    daily_data["precipitation_hours"] = daily_precipitation_hours
    daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
    daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
    daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
    daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum

    temp_df = pd.DataFrame(data = daily_data)
    daily_dataframe = pd.concat([daily_dataframe,temp_df])
    return daily_dataframe


In [6]:
df = get_weather_dataframe()

In [7]:
display(df)

,date,weather_code,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum
0,1980-01-01 00:00:00+00:00,3.0,-1.929167,1.200000,-4.0000,0.000000,0.000000,0.00,0.0,16.375053,28.440001,320.117371,2.58
1,1980-01-02 00:00:00+00:00,3.0,-2.406250,1.050000,-4.5500,0.000000,0.000000,0.00,0.0,16.981165,29.519999,314.023834,3.99
2,1980-01-03 00:00:00+00:00,71.0,2.079167,8.900000,-4.5000,7.199999,7.199999,0.14,15.0,23.863409,45.000000,194.125839,0.69
3,1980-01-04 00:00:00+00:00,3.0,7.283333,9.900000,2.9500,0.000000,0.000000,0.00,0.0,24.063513,44.639996,229.569687,1.75
4,1980-01-05 00:00:00+00:00,3.0,3.785417,7.050000,1.9500,0.000000,0.000000,0.00,0.0,22.322901,38.880001,252.675827,3.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14606,2019-12-28 00:00:00+00:00,3.0,7.802333,9.656500,6.3065,0.000000,0.000000,0.00,0.0,13.708391,23.759998,184.041275,1.77
14607,2019-12-29 00:00:00+00:00,3.0,5.929417,8.856501,3.4065,0.000000,0.000000,0.00,0.0,15.937878,27.359999,203.607742,3.47
14608,2019-12-30 00:00:00+00:00,3.0,5.448166,10.056500,2.2065,0.000000,0.000000,0.00,0.0,16.575644,28.440001,234.087982,3.77
14609,2019-12-31 00:00:00+00:00,3.0,5.952333,9.256500,2.2065,0.000000,0.000000,0.00,0.0,16.981165,28.440001,84.347328,1.49


In [8]:
df.to_csv('weather_output.csv')

In [9]:
df.describe()

,weather_code,temperature_2m_mean,temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum
count,14611.000000,14611.000000,14611.000000,14611.000000,14611.000000,14611.000000,14611.000000,14611.000000,14611.000000,14611.000000,14611.000000,14611.000000
mean,31.008076,10.508893,13.947825,6.953551,1.746643,1.709979,0.027629,3.445486,20.609449,41.773209,196.388062,10.457387
std,26.514736,5.517597,6.070994,5.206789,3.325738,3.291033,0.287403,4.692483,7.913365,15.703015,94.856796,7.255524
min,0.000000,-8.766666,-6.500000,-15.950000,0.000000,0.000000,0.000000,0.000000,4.735060,9.360000,0.000019,0.180000
25%,3.000000,6.373959,9.500000,3.000000,0.000000,0.000000,0.000000,0.000000,14.689589,30.599998,123.624100,3.955000
50%,51.000000,10.618751,13.850000,7.200000,0.100000,0.100000,0.000000,1.000000,19.592731,39.599998,219.142303,9.110000
75%,53.000000,14.942708,18.600000,11.150000,2.000000,2.000000,0.000000,6.000000,25.488914,50.399998,261.522858,16.000000
max,75.000000,27.521082,35.206501,19.850000,33.399998,33.399998,10.080001,24.000000,65.116226,131.040009,360.000000,29.840000
